In [1]:
import os 
from openai import OpenAI
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
client = OpenAI(
    api_key=OPENAI_API_KEY
)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


## Function for preprocessing

In [2]:
# load json file

import json

with open('./res/yanolja.json', 'r') as f:
    review_list = json.load(f)
    
review_list[:3]

[{'review': '직원분들 너무 친절하세요..\n데일리로 타올과 물을 챙겨주시고 좋았어요..\n\n근데 아쉬운 점 몇 가지 있었네요.\n데일리로 나온 쓰레기랑 이불정리는 해 주시는데 바닥 청소는 안 해 주셨더라구요.\n침실 불은 어둡고 이불은 조금만 풀썩거리면 털이 날라다녀서 알레르기 있는 식구들은 밤새 기침을 했네요.\n욕실은 샤워칸 안에서 문 닫고 샤워하면 그나마 낫지만 아이들 씻긴다고 문을 열고 샤워시켰더니 세면대 쪽 물이 고여 쉽게 빠지질 않더라구요..\n아이랑 같이 휴가 보내러 갔다 잠결에 화장실 가다 놀랠 일이 생길 듯해서 욕실화로 싹 싹 긁어 청소 아닌 청소 했네요.',
  'stars': 4,
  'date': '2시간 전'},
 {'review': '1시30분  로비도착  가방만  맡기려고했는데  바로  객실  카드  득덤했어요  조식포함  객실했어요  로비앞  성수기  시즌  조식1만원에  할인행사  있어요  \n바닥이  카펫이고요  ~ 베개는  포근했어요 \n ~  조식  먹을만해요 음식떨어지면  바로바로  채워주고  직원들  성실  친절해요  극극성수기로~사람이 많았어요',
  'stars': 3,
  'date': '17시간 전'},
 {'review': '휴가철에  정말 가성비 너무 좋게 다녀왔어요.\n조식도 좋았고 다만 주차가 힘들었지만\n그래도  담에 제주 가면 다시 예약합니다.\n가격대비  가성비 최고입니다!',
  'stars': 5,
  'date': '1일 전'}]

In [3]:
# good review and bad review
## good review = 5 stars
## bad review <= 4 stars

good_cnt, bad_cnt = 0, 0

for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1
        
good_cnt, bad_cnt

(267, 153)

In [4]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        
reviews_bad[:3]

['[REVIEW_START]직원분들 너무 친절하세요..\n데일리로 타올과 물을 챙겨주시고 좋았어요..\n\n근데 아쉬운 점 몇 가지 있었네요.\n데일리로 나온 쓰레기랑 이불정리는 해 주시는데 바닥 청소는 안 해 주셨더라구요.\n침실 불은 어둡고 이불은 조금만 풀썩거리면 털이 날라다녀서 알레르기 있는 식구들은 밤새 기침을 했네요.\n욕실은 샤워칸 안에서 문 닫고 샤워하면 그나마 낫지만 아이들 씻긴다고 문을 열고 샤워시켰더니 세면대 쪽 물이 고여 쉽게 빠지질 않더라구요..\n아이랑 같이 휴가 보내러 갔다 잠결에 화장실 가다 놀랠 일이 생길 듯해서 욕실화로 싹 싹 긁어 청소 아닌 청소 했네요.[REVIEW_END]',
 '[REVIEW_START]1시30분  로비도착  가방만  맡기려고했는데  바로  객실  카드  득덤했어요  조식포함  객실했어요  로비앞  성수기  시즌  조식1만원에  할인행사  있어요  \n바닥이  카펫이고요  ~ 베개는  포근했어요 \n ~  조식  먹을만해요 음식떨어지면  바로바로  채워주고  직원들  성실  친절해요  극극성수기로~사람이 많았어요[REVIEW_END]',
 '[REVIEW_START]욕실문이랑 옷장문이 통합되어있는게 좀 불편했어여\n조명이 어두워서 화장하기 좀 힘들었구 주차타워인데 미리 차를 뺄 수있는 시스템이 있으면 좋겠어요\n그 외로는 다 만족했습니다[REVIEW_END]']

In [5]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]직원분들 너무 친절하세요..\n데일리로 타올과 물을 챙겨주시고 좋았어요..\n\n근데 아쉬운 점 몇 가지 있었네요.\n데일리로 나온 쓰레기랑 이불정리는 해 주시는'

In [6]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/yanolja.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REIVEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
            
    
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    
    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:1000]

'[REIVEW_START]휴가철에  정말 가성비 너무 좋게 다녀왔어요.\n조식도 좋았고 다만 주차가 힘들었지만\n그래도  담에 제주 가면 다시 예약합니다.\n가격대비  가성비 최고입니다![REVIEW_END]\n[REIVEW_START]스탭들 진절하고 위치도 좋습니다.[REVIEW_END]\n[REIVEW_START]저렴하게 이용해서 좋았러요[REVIEW_END]\n[REIVEW_START]친절하시고 에어컨도 매번 청소하시는지 청소날짜 적혀있더라구요 기대 이상으로 깔끔했고 교통도 좋아요 조식도 한끼! 괜찮았어요[REVIEW_END]\n[REIVEW_START]올레시장 근처여서 접근성이 좋아요!! 객실 상태도 깨끗하고 매우 좋아요!!조식도 가격대비 맛이 좋더라구요~주차는 주차타워 이용하였는데, 부족시 근처 주차장 이용해야해요!!~[REVIEW_END]\n[REIVEW_START]침실 넓직하니 좋았습니다\n변기 청소상태가 안좋아서 이야기 드리니 바로 청소해주시고~서비스 만족입니다[REVIEW_END]\n[REIVEW_START]여러분 여기 꼭 이용해주세요!!\n\n1. 데스크 직원분들이 모두 하나같이 친절 그 자체였어요!\n\n2. 주차가 협소하긴하나 도보5분거리에 주차장있고 (**주차비 무료**)\n\n3. 오는정 김밥 바로 옆이에요~!\n\n4. 숙소 깨끗하고, 뷰가 좋았습니당!![REVIEW_END]\n[REIVEW_START]주차에서 프론트, 객실, 조식까지 아무런 불편없이 편하게 잘 지내고 왔습니다\r\n모두들 친절하시고 좋아요^^[REVIEW_END]\n[REIVEW_START]숙소 깔끔하도 좋았어요!!\n근데 suv차량은 호텔에 주차가 안돼서 3분거리 주차장에 주차를 해야해서 조금 불편했습니다 ㅠㅠ[REVIEW_END]\n[REIVEW_START]카운터 직원분들, 주차요원분들 다들 너무 친절하셨어요 :) 조식이 맛있어서 특히 좋았습니다!![REVIEW_END]\n[REIVEW_START]깨끗하고 프런트도 넘 친절하십미다[REVIEW_END]\n[RE

## 평가용 함수 작성

### 평가 기준 설명

- MT-Bench 논문 기반 Pariwise Comparison (=LLM 기반 평가)
    - 비교하는 방식 vs. 점수 매기는 방식
    - 점수라는게 애매할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

### 평가 스크립트 작성'
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [7]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""
        [System]
        Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
        AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
        follows the user’s instructions and answers the user’s question better. Your evaluation
        should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
        and level of detail of their responses. Begin your evaluation by comparing the two
        responses and provide a short explanation. Avoid any position biases and ensure that the
        order in which the responses were presented does not influence your decision. Do not allow
        the length of the responses to influence your evaluation. Do not favor certain names of
        the assistants. Be as objective as possible. After providing your explanation, output your
        final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
        if assistant B is better, and "[[C]]" for a tie.
        [User Reviews]
        {reviews}
        [The Start of Assistant A’s Answer]
        {answer_a}
        [The End of Assistant A’s Answer]
        [The Start of Assistant B’s Answer]
        {answer_b}
        [The End of Assistant B’s Answer]
    """
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )
    
    return completion

## Baseline 모델 개발

In [8]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [9]:
reviews, _ = preprocess_reviews(path='./res/yanolja.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews
    completion = client.chat.completions.create(
        model=model,
        messages=[{'role':'user', 'content':prompt}],
        temperature=temperature
    )
    
    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)


다양한 리뷰 중에서 가성비 좋고 깨끗한 숙소로 위치가 좋고 조식도 만족스러웠던 숙소로 추천되는 것 같다.


In [10]:
summary_good_real_20240803 = "직원들의 친절한 서비스와 깨끗한 숙소가 인상적입니다. 주변 맛집에 접근성이 좋고, 조식은 가격대비 만족스럽다는 후기가 많이 있습니다. 방은 깔끔하고 침구류는 편안하며, 주차는 주변 주차장을 이용해야 할 수도 있습니다. 일부 방에는 소음이 발생할 수 있으니 참고하시기 바랍니다. ganz.arso님에게 적합한 숙박지일 수 있습니다."
summary_bad_real_20240803 = "위치가 편리하고 직원들의 친절한 서비스가 인상적인 숙소입니다. 하지만 주차장이 불편하고 객실의 시설 상태가 약간 오래되어 있는 점이 아쉬운 부분이라는 후기가 있습니다. 또한 몇몇 객실에서는 환기가 필요한 냄새가 난다는 언급이 있습니다. 전체적으로 나쁘지 않지만 일부 개선이 필요한 숙소로 평가됩니다."

In [11]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_good_real_20240803).choices[0].message.content)

Both assistants provided summaries of the user reviews, but there are notable differences in the quality and comprehensiveness of their responses.

**Assistant A:**
- Provided a very brief summary.
- Highlighted the positive aspects such as good value for money, cleanliness, good location, and satisfactory breakfast.
- Lacked depth and detail, missing out on several key points mentioned in the reviews.

**Assistant B:**
- Offered a more detailed summary.
- Mentioned the friendly service, cleanliness, and good access to nearby restaurants.
- Included information about the breakfast being good value for money, the comfort of the rooms, and the potential need to use nearby parking.
- Noted the possibility of noise in some rooms, which is a critical detail for potential guests.

**Evaluation:**
- **Helpfulness:** Assistant B provided more useful information for potential guests, including both positive and negative aspects.
- **Relevance:** Both assistants stayed relevant to the reviews, b

## 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [12]:
eval_count = 10

summmaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summmaries_baseline

['주변 시장과의 접근성이 좋고, 깨끗하고 친절한 숙소. 가격대비 가성비 최고 수준이며, 조식도 만족스러웠던 리뷰입니다.',
 '다양한 숙소 리뷰 중에서 주로 가성비가 좋고 깨끗하며 위치가 좋다는 긍정적인 평가가 많이 있습니다. 일부 고객들은 주차에 불편함을 느꼈지만 전반적으로 만족하며 재방문을 희망하는 의견이 많이 보입니다.',
 '다양한 리뷰 중, 휴가철에 가성비 좋은 숙소라는 점이 강조되며 주차가 조금 어려웠지만 깔끔한 시설과 친절한 스탭으로 만족스러운 숙박 경험을 한다는 내용을 담고 있습니다. 다음 제주 방문 때 다시 예약할 의향이 있는 리뷰가 많이 보입니다.',
 '다양한 리뷰를 종합하여 이 숙소는 가성비가 매우 좋으며, 주차가 조금 불편하다는 의견이 있지만 위치가 좋고 깨끗한 숙소로, 조식도 맛있고 직원들도 친절하다는 내용이 담겨있다. 만족도가 높아 재방문을 고려하는 사람들이 많다.',
 '1. 휴가철에 가성비 좋게 머물기 좋은 곳.\n2. 주차가 조금 힘들었지만 다음에도 예약 의사 있음.\n3. 위치가 좋고 직원들 친절.\n4. 한라산 뷰가 최고고 조식도 맛있어요.\n5. 가성비 최고의 숙소로 만족스러운 여행했음.',
 '1. 휴가철에 최고의 가성비를 경험한 후 재방문 의사가 있는 숙소이며 조식도 칭찬했다.\n2. 위치가 좋고 직원들이 친절한 숙소이며 가격대비 수용이었다.\n3. 친절한 직원들과 깔끔한 숙소, 그리고 위치가 좋았다.\n4. 주변 맛집이 많고 깨끗한 숙소인데 주차가 불편한것이 단점.\n5. 뷰, 친절한 스탭, 조식까지 좋은 가성비가 돋보이는 숙소였다.',
 '다양한 숙소 리뷰를 요약해보면, 대부분의 손님들이 깨끗하고 가성비 좋은 숙소에 만족하며, 주변 시설과 접근성을 칭찬합니다. 몇몇 리뷰는 주차 불편을 언급하지만, 대부분의 손님들이 다시 방문 의사를 밝히며 친절한 직원과 좋은 서비스를 강조합니다. ',
 '1. 휴가철에 정말 가성비 좋게 다녀온 후 다음에 또 예약할 예정이며, 직원 친절과 깨끗한 시설을 칭찬합니다.\n2. 스탭은 진

In [13]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content
        
        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')
            
    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summmaries_baseline, [summary_good_real_20240803 for _ in range(len(summmaries_baseline))])
print(f'Wins: {wins}. Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:03<00:00,  6.30s/it]

Wins: 2. Losses: 8, Ties: 0


## 모델 고도화 1 - 조건들 명시

In [14]:
prompt = f"""
당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:
"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_good_real_20240803 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:29<00:00,  8.91s/it]

Wins: 7, Losses: 3, Ties: 0


## 모델 고도화 2 - 입력 데이터의 품질 증가

In [15]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/yanolja.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
        
        reviews_good, reviews_bad = [], []
        
        current_date = datetime.datetime.now()
        date_boundary = current_date - datetime.timedelta(days=6*30)
        
        filtered_cnt = 0
        
        for r in review_list:
            review_date_str = r['date']
            try:
                review_date = parser.parse(review_date_str)
            except (ValueError, TypeError):
                review_date = current_date
                
            if review_date < date_boundary:
                continue
            if len(r['review']) < 30 or len(r['review']) > 3000:
                # review length exceeds the limit of gpt model
                # BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 19172 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

                filtered_cnt += 1
                continue
            
            if r['stars'] == 5:
                reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
            else:
                reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
                
        reviews_good = reviews_good[:min(len(reviews_good), 50)]
        reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]
        
        reviews_good_text = '\n'.join(reviews_good)
        reviews_bad_text = '\n'.join(reviews_bad)
        
        return reviews_good_text, reviews_bad_text
    
reviews, _ = preprocess_reviews()

In [16]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_bad_real_20240803 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:53<00:00,  5.37s/it]

Wins: 9, Losses: 1, Ties: 0


## 모델 고도화 3 - Few-shot prompting

In [19]:
import crawler as cw
import importlib
importlib.reload(cw)

# crawler = cw.Crawler()
# crawler.crawl_yanolja_reviews('ninetree_pangyo', 'https://www.yanolja.com/reviews/domestic/1000113873')
# crawler.crawl_yanolja_reviews('ninetree_yongsan', 'https://www.yanolja.com/reviews/domestic/1000113873')

# crawling from other hotels
crawler_pangyo = cw.Crawler('ninetree_pangyo', 'https://www.yanolja.com/reviews/domestic/1000113873?sort=created-at%3Adesc')
crawler_yonsan = cw.Crawler('ninetree_yongsan', 'https://www.yanolja.com/reviews/domestic/10048873?sort=created-at%3Adesc')
crawler_pangyo.crawl_yanolja_reviews()
crawler_yonsan.crawl_yanolja_reviews()

In [20]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_good_real_20240803 for _ in range(len(summaries))])

print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]

Wins: 8, Losses: 2, Ties: 0


In [21]:
summaries

['서귀포에 위치한 숙소는 휴가철에 가성비가 좋다는 평가를 받았습니다. 주차는 조금 힘들었지만, 다음에도 다시 예약할 의향이 있는 만큼 가성비는 최상입니다. 또한, 객실 상태가 깨끗하고 조식도 좋다는 평을 받았습니다. 또한, 룸 업그레이드와 친절한 직원들이 인상깊었던 숙소는 재방문의사가 높은 곳으로 평가되었습니다. 특히 조식이 맛있어서 특별히 언급되는 곳도 있습니다. 반면 주차 및 방음에 있어 불편함을 언급하는 리뷰도 있으나 전반적으로 만족도가 높은 숙소로 요약됩니다.',
 '서귀포 여행객들이 가성비 좋은 숙박을 찾으실 때 이용한 숙소로, 객실 상태가 깨끗하고 직원들이 친절했다는 평가를 받았습니다. 주변에 오는정 김밥과 올레시장이 가까워서 식사나 쇼핑하기에도 편리했고, 주차도 편리했습니다. 다음에 서귀포 방문 시 또 이용하고 싶은 숙소로 추천받았습니다. 특히 조식이 맛있었다고 사람들이 호평했으며, 숙소 전체적으로 깔끔하고 조용한 분위기를 즐기기에 좋은 곳으로 평가받았습니다.',
 '1. 휴가철에 가성비 좋은 숙소로 다녀왔습니다. 주차가 힘들었지만, 담에 제주를 다시 방문할 예정입니다. \n\n2. 직원분들이 친절하고 청결한 객실, 교통이 편리한 숙소입니다. 조식도 괜찮았습니다. 오는정김밥이 가까이에 있어 굿굿!\n\n3. 침실이 넓고 변기 청소도 바로 처리해주는 서비스에 만족하였습니다. 청결한 숙소와 좋은 서비스 였어요.\n\n4. 데스크 직원부터 객실, 조식까지 편안하게 지내다 왔습니다. 모두 친절하고 좋았어요^^\n\n5. 위치가 좋고 깨끗한 숙소에서 편하게 쉬었습니다. 특히 아침 조식이 맛있었습니다. 재방문 의사가 있어요.',
 '1. 휴가철에 방문한 숙소는 가성비가 좋았고, 조식도 매력적이었습니다. 주차가 조금 어렵지만, 다음에 다시 예약할 예정입니다.\n2. 친절한 직원들과 청결한 객실로 인상적인 숙박 경험을 제공받았습니다. 교통이 편리하고 조식도 만족스러웠어요.\n3. 올레시장 근처에 위치한 숙소는 접근성이 좋았고 객실 상태가 매우 좋았습니다. 조식

In [22]:
review_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(review_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

In [23]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_good_real_20240803 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:42<00:00,  4.22s/it]

Wins: 7, Losses: 3, Ties: 0
